In [1]:
import math
import random
import string
import re 
from  collections import Counter
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
from scipy.spatial.distance import cdist
import statistics
import scipy.stats as sps
from sklearn.model_selection import train_test_split

## Add zip code to each row in restaurant data

In [2]:
data = pd.read_csv("../datasets/restaurant_data.csv")

In [3]:
data

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,s9pxTIWRwCHzMAOHbcmuFw,chicken-shack-astoria,Chicken Shack,https://s3-media2.fl.yelpcdn.com/bphoto/h4s59I...,False,https://www.yelp.com/biz/chicken-shack-astoria...,109,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 40.75991227948, 'longitude': -73....","['pickup', 'delivery']",$$,"{'address1': '36-19 Broadway', 'address2': '',...",1.718721e+10,(718) 721-3035,10936.818787
1,e08qVeVYBR-0QI_G0NDZpw,sugar-monk-new-york,Sugar Monk,https://s3-media1.fl.yelpcdn.com/bphoto/mMpmMn...,False,https://www.yelp.com/biz/sugar-monk-new-york?a...,152,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 40.80896, 'longitude': -73.95184}","['delivery', 'pickup']",$$,"{'address1': '2292 Frederick Douglass Blvd', '...",1.917409e+10,(917) 409-0028,1144.427093
2,a-hVnSTC12MY5L3t9yOSJw,eastern-chinese-restaurant-long-island-city,Eastern Chinese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/wwv8uG...,False,https://www.yelp.com/biz/eastern-chinese-resta...,31,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,"{'latitude': 40.7656686822612, 'longitude': -7...","['delivery', 'pickup']",$,"{'address1': '2105 Broadway', 'address2': '', ...",1.718279e+10,(718) 278-7333,86.164334
3,JWQlPP4Oxrtwb0rNjnVgIg,hunts-point-chinese-restaurant-bronx,Hunts Point Chinese Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/Gr5yzW...,False,https://www.yelp.com/biz/hunts-point-chinese-r...,8,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 40.81965, 'longitude': -73.89302}",['delivery'],NaN,"{'address1': '901 Southern Blvd', 'address2': ...",1.718378e+10,(718) 378-2055,21.323893
4,nXh79laNQ7leN92F5Ore-Q,forever-taste-new-york,Forever Taste,https://s3-media2.fl.yelpcdn.com/bphoto/ovt1wc...,False,https://www.yelp.com/biz/forever-taste-new-yor...,52,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 40.71293, 'longitude': -73.98994}",['delivery'],$,"{'address1': '27 Rutgers St', 'address2': '', ...",1.646289e+10,(646) 288-8434,25.210535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4690,ziIb68Res3eMw7fz83yFxw,two-hands-brooklyn,Two Hands,https://s3-media4.fl.yelpcdn.com/bphoto/ldmjoA...,False,https://www.yelp.com/biz/two-hands-brooklyn?ad...,51,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",3.5,"{'latitude': 40.716374709082125, 'longitude': ...","['delivery', 'pickup']",$$,"{'address1': '262 Kent Ave', 'address2': '', '...",NaN,NaN,18.335996
4691,csMJyqX4hwcP-VzvuDH3Bw,dunkin-donuts-baskin-robbins-jamaica-2,Dunkin' Donuts Baskin Robbins,https://s3-media1.fl.yelpcdn.com/bphoto/sQ7nU0...,False,https://www.yelp.com/biz/dunkin-donuts-baskin-...,7,"[{'alias': 'desserts', 'title': 'Desserts'}]",2.5,"{'latitude': 40.706409662962, 'longitude': -73...",['delivery'],$,"{'address1': '168-04 Jamaica Ave', 'address2':...",NaN,NaN,14775.461860
4692,qEXpXF4p8uM1BfqV5tdYqg,darbars-chicken-and-ribs-queens-2,Darbars Chicken & Ribs,https://s3-media2.fl.yelpcdn.com/bphoto/arE0Dc...,False,https://www.yelp.com/biz/darbars-chicken-and-r...,139,"[{'alias': 'halal', 'title': 'Halal'}, {'alias...",3.5,"{'latitude': 40.688737, 'longitude': -73.8181408}","['delivery', 'pickup']",$,"{'address1': '12609 Liberty Ave', 'address2': ...",1.718529e+10,(718) 529-4900,17.159259
4693,Qli775QfVY4oFPNu7Cy5Tw,yakitori-sun-chan-new-york,Yakitori Sun-Chan,https://s3-media4.fl.yelpcdn.com/bphoto/RMtHY6...,False,https://www.yelp.com/biz/yakitori-sun-chan-new...,255,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 40.7995, 'longitude': -73.9686399}",['delivery'],$$,"{'address1': '2707 Broadway', 'address2': None...",1.212750e+10,(212) 749-5008,29.316169


In [4]:
new_col = [eval(data["location"][i])["zip_code"] for i in range(0, 4695)]

In [5]:
data["zip_code"] = new_col

In [6]:
result = data[['id', 'name', "review_count", "rating", "price", "location", "zip_code", "url"]]

In [7]:
result

,id,name,review_count,rating,price,location,zip_code,url
0,s9pxTIWRwCHzMAOHbcmuFw,Chicken Shack,109,3.0,$$,"{'address1': '36-19 Broadway', 'address2': '',...",11106,https://www.yelp.com/biz/chicken-shack-astoria...
1,e08qVeVYBR-0QI_G0NDZpw,Sugar Monk,152,4.5,$$,"{'address1': '2292 Frederick Douglass Blvd', '...",10027,https://www.yelp.com/biz/sugar-monk-new-york?a...
2,a-hVnSTC12MY5L3t9yOSJw,Eastern Chinese Restaurant,31,3.0,$,"{'address1': '2105 Broadway', 'address2': '', ...",11106,https://www.yelp.com/biz/eastern-chinese-resta...
3,JWQlPP4Oxrtwb0rNjnVgIg,Hunts Point Chinese Restaurant,8,4.0,NaN,"{'address1': '901 Southern Blvd', 'address2': ...",10459,https://www.yelp.com/biz/hunts-point-chinese-r...
4,nXh79laNQ7leN92F5Ore-Q,Forever Taste,52,4.0,$,"{'address1': '27 Rutgers St', 'address2': '', ...",10002,https://www.yelp.com/biz/forever-taste-new-yor...
...,...,...,...,...,...,...,...,...
4690,ziIb68Res3eMw7fz83yFxw,Two Hands,51,3.5,$$,"{'address1': '262 Kent Ave', 'address2': '', '...",11249,https://www.yelp.com/biz/two-hands-brooklyn?ad...
4691,csMJyqX4hwcP-VzvuDH3Bw,Dunkin' Donuts Baskin Robbins,7,2.5,$,"{'address1': '168-04 Jamaica Ave', 'address2':...",11432,https://www.yelp.com/biz/dunkin-donuts-baskin-...
4692,qEXpXF4p8uM1BfqV5tdYqg,Darbars Chicken & Ribs,139,3.5,$,"{'address1': '12609 Liberty Ave', 'address2': ...",11419,https://www.yelp.com/biz/darbars-chicken-and-r...
4693,Qli775QfVY4oFPNu7Cy5Tw,Yakitori Sun-Chan,255,3.5,$$,"{'address1': '2707 Broadway', 'address2': None...",10025,https://www.yelp.com/biz/yakitori-sun-chan-new...


In [8]:
result.to_csv('../datasets/with_zip_code.csv')